In [1]:
# input: url string
# output: bytes

In [13]:
import os
import time
import pickle
import collections
import gzip

import tqdm

In [3]:
hash_log_filepath = 'tmp_pickle_files/hash_log-1650109654532523300.pickle'
os.path.getsize(hash_log_filepath)

20001503

In [4]:
with open(hash_log_filepath, mode='rb') as infile:
    hash_log = pickle.load(infile)
    print(type(hash_log))

<class 'list'>


In [6]:
len(hash_log)

195094

In [5]:
hash_log[0]

{'path': '.requests_cache\\00\\000000a86ac9fd0cc77dc4ecf80f0b89',
 'size': 132302,
 'md5hash': b"\t\xf9V!W\x82\xbcIt/\xcc\x8c'k\x1f\xe1"}

# store unique content by md5 hash and size

In [9]:
# key: (md5, size)
# value: list of old url md5 hash filepath

new_cache_structure_content_dict = collections.defaultdict(list)

invalid_hash_log_entries = []

for hash_log_entry in tqdm.tqdm(hash_log):
    if not isinstance(hash_log_entry, dict):
        invalid_hash_log_entries.append(hash_log_entry)
        continue

    if 'path' not in hash_log_entry:
        invalid_hash_log_entries.append(hash_log_entry)
        continue

    if 'size' not in hash_log_entry:
        invalid_hash_log_entries.append(hash_log_entry)
        continue

    if 'md5hash' not in hash_log_entry:
        invalid_hash_log_entries.append(hash_log_entry)
        continue

    new_cache_structure_content_dict[(hash_log_entry['md5hash'], hash_log_entry['size'])].append(hash_log_entry['path'])

key_list = list(new_cache_structure_content_dict.keys())
len(key_list)

100%|██████████| 195094/195094 [00:00<00:00, 305485.96it/s]


133089

In [12]:
new_cache_structure_content_dict_log_filepath = f'new_cache_structure_content_dict-{time.time()}.pickle'
print(new_cache_structure_content_dict_log_filepath)

with open(new_cache_structure_content_dict_log_filepath, mode='wb') as outfile:
    pickle.dump(new_cache_structure_content_dict, outfile)

new_cache_structure_content_dict-1653545293.578412.pickle


In [10]:
len(invalid_hash_log_entries)

0

## copy the body content to another directory

In [14]:
key_list[0]

(b"\t\xf9V!W\x82\xbcIt/\xcc\x8c'k\x1f\xe1", 132302)

In [16]:
body_content_root_dir = 'd:\htmlbodycontentcache'
if not os.path.exists(body_content_root_dir):
    os.makedirs(body_content_root_dir)

In [18]:
deleted_cache_files = []

pbar = tqdm.tqdm(key_list)
for key in pbar:
    if os.path.exists('stop'):
        break

    cache_filepath_list = new_cache_structure_content_dict[key]
    md5hash_bs, size = key

    if size == 0:
        continue

    md5hash_str = md5hash_bs.hex()
    new_cache_filename = f'{md5hash_str}-{size}.gzip'
    new_cache_filepath = os.path.join(body_content_root_dir, new_cache_filename)
    if os.path.exists(new_cache_filepath):
        continue

    pbar.set_description(new_cache_filename)

    for old_cache_filepath in cache_filepath_list:
        if not os.path.exists(old_cache_filepath):
            deleted_cache_files.append(old_cache_filepath)
            continue

        with gzip.open(new_cache_filepath, mode='wb') as outfile:
            with open(old_cache_filepath, mode='rb') as infile:
                content_bs = infile.read()
                outfile.write(content_bs)

9fb27cab19d68f8a63f632cc0475ef9c-802344.gzip: 100%|██████████| 133089/133089 [1:46:55<00:00, 20.74it/s]     


# old code

In [ ]:
url_info_list_filepath = 'tmp_pickle_files/url_info_list-1650285069904349100'
os.path.getsize(url_info_list_filepath)

In [ ]:
with open(url_info_list_filepath, mode='rb') as infile:
    url_info_list = pickle.load(infile)
    print(type(url_info_list))

In [ ]:
len(url_info_list)

In [ ]:
url_info_list[0]

In [ ]:
import tqdm

In [ ]:
number_of_url_info = len(url_info_list)
number_of_url_info

In [ ]:
for index in tqdm.tqdm(range(number_of_url_info)):
    url_info = url_info_list[index]
    if 'response_size' not in url_info:
        print(index)

In [ ]:
for url_info in tqdm.tqdm(url_info_list):
    response_size = url_info['response_size']
    cache_path = url_info['cache_path']
    if response_size == 0:
        os.remove(cache_path)

In [ ]:
import traceback

In [ ]:
for index in tqdm.tqdm(range(number_of_url_info)):
    url_info = url_info_list[index]
    try:
        response_size = url_info['response_size']
        cache_path = url_info['cache_path']
        if response_size == 0:
            if os.path.exists(cache_path):
                os.remove(cache_path)
    except Exception as ex:
        stack_trace_str = traceback.format_exc()
        print(stack_trace_str)
        print(index)
        print(url_info)
        print(ex)

In [ ]:
url_info_list[9158]

In [ ]:
os.path.exists('.requests_cache\\fc\\fc2cabf6e440dcc364991de186d76067')